# Explications de la fonction : apply_vector_field_forward

**Signature** : apply_vector_field_forward(input, vector_field, up, down, left, right)    
**Docstring** : extrapole l'image `input` en utilisant le champ de vecteur `vector_field`. Ne change l'image qu'entre [`up`:`down`, `left`:`right`]

## Explications théoriques
Nous allons appliquer un champ de vecteurs à une image. Pour cela, nous utilisons une méthode décrite par George Wolberg dans son livre "Digital image warping". Au chapitre 3 (Spatial transformations), il nous parle de forward mapping, qui consiste à partir de l'image d'entrée et à définir pour chaque pixel sa future position (par opposition à l'inverse mapping où on part de la sortie en cherchant dans l'entrée). Afin de résoudre certains problèmes qui apparaissent, Wolberg nous expose une technique de forward mapping : le four-corner mapping. Au lieu de considérer les pixels eux-mêmes, nous allons considérer leurs 4 coins, et appliquer le champ de vecteurs sur ces coins, ce qui nous permet d'obtenir une image parfaitement continue en sortie.  
Cependant, comme nous restons sur une image matricielle en sortie, il faut appliquer ces pixels modifiés sur une grille de pixels "normaux". Pour cela, nous devons calculer, pour chaque pixel modifié, la surface de pixels normaux qu'il recouvre. Ainsi pour chaque pixel normal, on peut obtenir sa couleur finale en additionnant les contributions de chaque pixel modifié. On obtient :  
$$O_{i'j'} = \sum_{i=0}^{height}\sum_{j=0}^{width}I_{ij}*w(i,j,i',j')$$
avec $O_{i'j'}$ la valeur de l'image de sortie en (i',j'), $I_{ij}$ la valeur de l'image d'entrée en (i,j) et $w(i,j,i',j')$ une fonction qui nous renvoie le pourcentage du pixel normal (i',j') recouvert par le pixel (i,j) modifié.

![Schema_pixels](image_notebook/schema_final_pixels.png)

On peut voir sur le schéma un exemple d'application d'un champ quelconque sur une grille de pixels. On modifie la position des coins des pixels (on ne touche cependant pas aux bords de l'image/la zone à modifier) ce qui crée de nouveaux quadrilatères. Ces nouvelles figures recouvrent de différentes manières les pixels normaux. On peut voir que le pixel modifié vert ne recouvre même plus un pixel entier alors que le pixel noir s'étale sur plusieurs autres pixels. Pour obtenir la couleur du pixel en haut à gauche, on additionnera environ 90% de la couleur initiale du pixel jaune et 10% de la couleur du pixel rouge.

## Importation des modules

In [ ]:
import numpy as np # pour les opérations sur l'image et la manipulation de tableaux
from tqdm import tqdm, trange # pour afficher des barres de progression
from shapely import Polygon, area, intersection, is_valid # opérations sur les pixels, calcul d'intersection...
from math import ceil
from time import perf_counter # pour mesurer le temps de chaque opération
from multiprocessing import Pool # pour utiliser le multiprocessing
import matplotlib.pyplot as plt # pour afficher les images et les vecteurs
from math import atan, cos, sin, exp, pi, sqrt, log

## Choix de l'image d'entrée

In [ ]:
# on choisit une image sur laquelle appliquer notre champ
img_now = plt.imread("data_verification/msg_initial/tif/3kmVIS006_msg03_202401011200.tif")[1500:2000, 1500:2000]

plt.rcParams['figure.figsize'] = [5, 5]
plt.imshow(img_now, cmap="gray")
plt.title("Image d'entrée")
plt.show()

## Création du champ de vecteurs

Nous avons créé 2 champs différents, un champ radial et un champ plus variable avec des convergences et divergences. Vous pouvez choisir un des deux en exécutant sa cellule.

### Champ radial

Nous allons créer un champ de vecteurs radial pour vérifier notre algorithme. Celui vaudra 0 au centre (250, 250), puis la norme de chaque vecteur commencera à 20 autour du centre et diminuera de façon exponentielle :
$$||\vec v|| = 20*e^{-(\frac{r^2}{\delta})^2}$$
avec : $$r^2 = (row-center\_row)^2 + (col-center\_col)^2$$
$$\delta = \frac{\min(width, height)^2}{4\sqrt{-\ln{\frac{norme\_fin}{norme}}}}$$
Le coefficient $\delta$ nous permet de définir l'évolution de la norme et donc sa valeur au bord de l'image. Ici nous avons choisi d'obtenir une norme de $10^{-6}$ quand nous atteignons le bord.


In [ ]:
size = len(img_now)
vector_field = np.zeros((2, size, size))
center = size // 2
norme = 20
norme_fin = 10**(-6)
delta = size**2 // (4*sqrt(-log(norme_fin/norme)))

plt.rcParams['figure.figsize'] = [20, 5]

fig, axes = plt.subplots(1, 2)

axes[0].imshow(plt.imread("image_notebook/champ_radial.png"))
axes[0].axis('off')
axes[0].title.set_text("Schéma du champ de vecteurs souhaité")

axes[1].plot(np.arange(center), norme*np.exp(-(np.arange(center)**2/delta)**2))
axes[1].title.set_text("Evolution de la norme des vecteurs en fonction du rayon")

plt.show()

rows, cols = np.meshgrid(np.arange(size), np.arange(size), indexing="ij")

def expo_norme(r, c):
    r = (r-center)**2 + (c-center)**2
    return norme*exp(- (r / delta)**2)

for row in trange(size):
    for col in range(size):
        if row == center:
            dr = 0
            dc = expo_norme(row, col) * np.sign(col - center)
        else:
            alpha = atan((col - center) / (row - center))
            if row-center < 0:
                if col-center < 0:
                    alpha -= pi
                else:
                    alpha += pi
            dr = expo_norme(row, col) * cos(alpha)
            dc = expo_norme(row, col) * sin(alpha)
        vector_field[0, row, col] = dr
        vector_field[1, row, col] = dc

### Champ variable avec dépressions et anticyclones

In [ ]:
rows, cols = np.meshgrid(np.arange(size), np.arange(size), indexing="ij")

vector_field[0] = norme*np.sin((rows+cols)*2*pi/size)
vector_field[1] = norme*np.cos((rows-cols)*2*pi/size)

### Affichage du champ de vecteurs choisi

In [ ]:
step_quiver = 25

plt.rcParams['figure.figsize'] = [5, 5]
plt.imshow(img_now, cmap="gray")
plt.quiver(cols[::step_quiver,::step_quiver],
           rows[::step_quiver,::step_quiver],
           vector_field[1, ::step_quiver, ::step_quiver],
           vector_field[0, ::step_quiver, ::step_quiver],
           color="red",
           angles="xy",
           scale_units="xy",
           scale=1)
plt.title("Représentation du champ de vecteurs par rapport à l'image d'entrée")
plt.show()

## Application du champ de vecteurs sur l'image d'entrée

In [ ]:
# variables globales pour que les sous-processus puissent y accéder facilement
# cela n'amène pas de copie car elles sont accédées en lecture
left = right = up = down = 0 # les limites dans lesquelles on applique le champ
width = height = 0 # dimensions de l'image d'entrée et donc de sortie
next_row = next_col = None # tableaux numpy qui indiquent pour chaque coin de pixel sa position dans l'image modifiée

### Création des array next_row et next_col

In [ ]:
# fonction qui va nous permettre de créer les tableaux next_row et next_col
def compute_next_pos(args):
    """
    Return the next row and col of args=(row,col) according
    to the next_row and next_col matrix

    Args:
        args: tuple (row, col)
    Return:
        (row,col,next_row,next_col): tuple
    """
    row, col = args # on est obligé de passer les paramètres comme ça à cause du multiprocessing
    dr,dc = vector_field[:,row,col]

    next_r = np.clip(row+dr, up, down, dtype=np.float64)
    next_c = np.clip(col+dc, left, right, dtype=np.float64)

    return row, col, next_r, next_c

In [ ]:

height, width = len(img_now), len(img_now[0]) 

up = 0
down = height
left = 0
right = width

# comme on n'applique le champ de vecteurs que sur une certaine zone, le reste sera égal à l'input
output = img_now.copy()
# on initialise à 0 ce qu'on va modifier
output[up:down, left:right] = np.zeros((down-up, right-left))

# +1 car on prend les coins des pixels, il y en a un de plus que le nombre de pixels
next_row, next_col = np.meshgrid(np.arange(height+1, dtype=np.float64),
                                    np.arange(width+1, dtype=np.float64),
                                    indexing="ij")

print("création des coordonnées...") # elles vont servir d'arguments pour les fonctions des sous-processus
temp = perf_counter()
row_coords, col_coords = np.meshgrid(np.arange(height), np.arange(width), indexing="ij")
print(f"=> {perf_counter() - temp} s")

argument = np.column_stack((row_coords[up+1:down-1, left+1:right-1].ravel(),
                            col_coords[up+1:down-1, left+1:right-1].ravel()))
# meshgrid nous donnait un array de la forme (2, height, width)
# grâce à column_stack, on obtient un array de la forme (width, height, 2)
# qui sera utilisable pour les fonctions 
    
chunksize = 1000 # nombre d'opérations données à chaque processus à chaque fois
# ce nombre trouvé de manière empirique est celui qui donne les meilleures performances

with Pool() as pool: # crée le plus de "workers" possibles = sous-processus
    print("calcul des nouvelles positions...")
    temp = perf_counter()
    # on crée les valeurs qui vont remplir les array next_row et next_col
    res_tuples = np.array(list(tqdm(pool.imap(compute_next_pos,
                                                argument, chunksize), total=len(argument))))
# et on les remplit avec les valeurs
# on est obligé de sortir de la pool pour actualiser la variable globale
# et ensuite refaire une pool pour qu'elle s'actualise dans tous les sous-processus
rows = res_tuples[:, 0].astype(np.uint32)
cols = res_tuples[:, 1].astype(np.uint32)

next_row[rows, cols] = res_tuples[:, 2]
next_col[rows, cols] = res_tuples[:, 3]
print(f"=> {perf_counter() - temp} s")

In [ ]:
# on peut visualiser les pixels modifiés
# on fait un zoom sur le centre de la figure
plt.rcParams['figure.figsize'] = [20, 10]
fig, axes = plt.subplots(1, 2)
step_scatter = 5
axes[0].scatter(col_coords[::step_scatter, ::step_scatter], row_coords[::step_scatter, ::step_scatter], s=1, c="b")
axes[0].title.set_text("Pixels normaux")
axes[0].invert_yaxis()

axes[1].scatter(next_col[::step_scatter, ::step_scatter], next_row[::step_scatter, ::step_scatter], s=1, c="r")
axes[1].title.set_text("Pixels modifiés")
axes[1].invert_yaxis()
plt.show()

### Application de la formule sur l'output

In [ ]:
# fonction qui permet, à partir de la ligne et la colonne d'un pixel, de trouver
# son pixel modifié associé, et le retourne sous la forme d'un polygone
# afin de faire des opérations de calcul d'intersection et d'aire ensuite
def calculate_moved_tile(row, col):
    """
    Return the moved tile according to the vector field in
    next_row and next_col

    Args:
        row: int
        col: int
    Return:
        moved_tile: Polygon
    """
    return Polygon(((next_row[row,col], next_col[row,col]),
                (next_row[row,col+1], next_col[row,col+1]),
                (next_row[row+1,col+1], next_col[row+1,col+1]),
                (next_row[row+1,col], next_col[row+1,col])))

In [ ]:
# fonction qui retourne, pour un pixel modifié, la liste de ses contributions
# aux futurs pixels normaux sous la forme d'une liste :
# [[future_row, future_col, contribution], [...], ...]
def calculate_coverage_moved_tile(args):
    """
    Compute the area of the intersection of a moved tile and the
    pixels it covers.

    Args:
        args: tuple (row, col)
    Return:
        list of intensities applied by the moved_tile[args] on a pixel: List[List[int, int, float]] (pixel_row, pixel_col, intensity)
    """
    row, col = args
    moved_tile = calculate_moved_tile(row, col)
    res = []
    if is_valid(moved_tile):
        bbox = moved_tile.bounds
        for interrow in range(int(bbox[0]), ceil(bbox[2])):
            for intercol in range(int(bbox[1]), ceil(bbox[3])):
                pixel = Polygon(((interrow, intercol),
                                 (interrow, intercol+1),
                                 (interrow+1, intercol+1),
                                 (interrow+1, intercol)))
                inter = intersection(pixel, moved_tile)
                coverage = area(inter)
                res.append((interrow, intercol, coverage*img_now[row,col]))
    return res

In [ ]:
argument = np.column_stack((row_coords[up:down, left:right].ravel(),
                            col_coords[up:down, left:right].ravel()))

with Pool() as pool:
    print("calcul des intersections...")
    temp = perf_counter()
    # on crée les valeurs qui vont remplir l'output final
    res_tuples = [t for t in tqdm(pool.imap(calculate_coverage_moved_tile,
                                            argument, chunksize), total=len(argument)) if t]
    all_results = np.concatenate(res_tuples)
    # concatenate permet de passer d'un array de la forme [[[1,2,3], [4,5,6]], [], [[7,8,9]]]
    # à un array [[1,2,3], [4,5,6], [7,8,9]]
    print(f"=> {perf_counter() - temp} s")

print("application des intersections...")
temp = perf_counter()
rows = all_results[:, 0].astype(np.uint32)
cols = all_results[:, 1].astype(np.uint32)
terms = all_results[:, 2].astype(np.uint8)
# on additionne les contributions de chaque pixel au bon endroit
np.add.at(output, (rows, cols), terms)
print(f"=> {perf_counter() - temp} s")

### Affichage des résultats

In [ ]:
plt.rcParams['figure.figsize'] = [20, 20]
fig, axes = plt.subplots(2, 2)

rows, cols = np.meshgrid(np.arange(size), np.arange(size), indexing="ij")

step_quiver = 25
axes[0,0].imshow(img_now, cmap="gray")
axes[0,0].quiver(cols[::step_quiver,::step_quiver],
           rows[::step_quiver,::step_quiver],
           vector_field[1, ::step_quiver, ::step_quiver],
           vector_field[0, ::step_quiver, ::step_quiver],
           color="red",
           angles="xy",
           scale_units="xy",
           scale=1)
axes[0,0].title.set_text("Image d'entrée avec le champ de vecteurs")

axes[0,1].imshow(output, cmap="gray")
axes[0,1].title.set_text("Image de sortie")

step_scatter = 10
axes[1,0].imshow(img_now, cmap="gray")
axes[1,0].scatter(col_coords[::step_scatter, ::step_scatter]-0.5, row_coords[::step_scatter, ::step_scatter]-0.5, s=1, c="b")
axes[1,0].title.set_text("Image d'entrée avec les coordonnées des coins des pixels")

axes[1,1].imshow(output, cmap="gray")
axes[1,1].scatter(next_col[::step_scatter, ::step_scatter]-0.5, next_row[::step_scatter, ::step_scatter]-0.5, s=1, c="r")
axes[1,1].title.set_text("Image de sortie avec les coordonnées des coins des pixels modifiés")

plt.show()